In [2]:
import json
import docsim
import re
import csv
import nltk
nltk.download('wordnet')
from urllib.request import urlopen
from urllib.error import HTTPError
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tfidf import rank_documents
from pathlib import Path
import random
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/neeldatta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/neeldatta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv("ASSESSMENT_TEST - output.csv", header = None)
df

0                                                  1  \
0      ac-2  Organizational processes account management on...   
1    ac-2.1  Automated mechanisms implementing account mana...   
2    ac-2.2  Automated mechanisms implementing account mana...   
3    ac-2.3  Automated mechanisms implementing account mana...   
4    ac-2.4  Automated mechanisms implementing account mana...   
..      ...                                                ...   
645   pm-12  Organizational processes for implementing the ...   
646   pm-13  Organizational processes for implementing info...   
647   pm-14  Organizational processes for development and m...   
648   pm-15  Organizational processes for establishing and ...   
649   pm-16  Organizational processes for implementing the ...   

                                                     2  \
0    automated mechanisms for implementing account ...   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN   
..                                                 ...   
645  automated mechanisms supporting and/or impleme...   
646  automated mechanisms supporting and/or impleme...   
647  automated mechanisms supporting development an...   
648  automated mechanisms supporting contacts with ...   
649  Organizational processes for implementing the ...   

                                                     3  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN   
..                                                 ...   
645                                                NaN   
646                                                NaN   
647                                                NaN   
648                                                NaN   
649  automated mechanisms supporting and/or impleme...   

                                                     4    5    6  
0                                                  NaN  NaN  NaN  
1                                                  NaN  NaN  NaN  
2                                                  NaN  NaN  NaN  
3                                                  NaN  NaN  NaN  
4                                                  NaN  NaN  NaN  
..                                                 ...  ...  ...  
645                                                NaN  NaN  NaN  
646                                                NaN  NaN  NaN  
647                                                NaN  NaN  NaN  
648                                                NaN  NaN  NaN  
649  automated mechanisms supporting and/or impleme...  NaN  NaN  

[650 rows x 7 columns]

In [4]:
df = df[~df[0].str.contains('\*', regex = True)]

In [5]:
df

0                                                  1  \
0      ac-2  Organizational processes account management on...   
1    ac-2.1  Automated mechanisms implementing account mana...   
2    ac-2.2  Automated mechanisms implementing account mana...   
3    ac-2.3  Automated mechanisms implementing account mana...   
4    ac-2.4  Automated mechanisms implementing account mana...   
..      ...                                                ...   
645   pm-12  Organizational processes for implementing the ...   
646   pm-13  Organizational processes for implementing info...   
647   pm-14  Organizational processes for development and m...   
648   pm-15  Organizational processes for establishing and ...   
649   pm-16  Organizational processes for implementing the ...   

                                                     2  \
0    automated mechanisms for implementing account ...   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN   
..                                                 ...   
645  automated mechanisms supporting and/or impleme...   
646  automated mechanisms supporting and/or impleme...   
647  automated mechanisms supporting development an...   
648  automated mechanisms supporting contacts with ...   
649  Organizational processes for implementing the ...   

                                                     3  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN   
..                                                 ...   
645                                                NaN   
646                                                NaN   
647                                                NaN   
648                                                NaN   
649  automated mechanisms supporting and/or impleme...   

                                                     4    5    6  
0                                                  NaN  NaN  NaN  
1                                                  NaN  NaN  NaN  
2                                                  NaN  NaN  NaN  
3                                                  NaN  NaN  NaN  
4                                                  NaN  NaN  NaN  
..                                                 ...  ...  ...  
645                                                NaN  NaN  NaN  
646                                                NaN  NaN  NaN  
647                                                NaN  NaN  NaN  
648                                                NaN  NaN  NaN  
649  automated mechanisms supporting and/or impleme...  NaN  NaN  

[620 rows x 7 columns]

In [6]:
queries = df[1]

In [7]:
len(queries)

620

In [8]:
queries

0      Organizational processes account management on...
1      Automated mechanisms implementing account mana...
2      Automated mechanisms implementing account mana...
3      Automated mechanisms implementing account mana...
4      Automated mechanisms implementing account mana...
                             ...                        
645    Organizational processes for implementing the ...
646    Organizational processes for implementing info...
647    Organizational processes for development and m...
648    Organizational processes for establishing and ...
649    Organizational processes for implementing the ...
Name: 1, Length: 620, dtype: object

In [9]:
randqueries = []
random.seed(10)
for i in range(20):
    randqueries.append(queries[random.randint(0, 619)])
randqueries

['Organizational processes for intrusion detection/information system monitoring',
 'Automated mechanisms implementing information flow enforcement policy',
 'Organizational processes for re-implementing or customized development of critical information system components',
 'Automated mechanisms supporting and/or implementing physical disconnect of collaborative computing devices',
 'Organizational processes for information system monitoring',
 'Automated mechanisms implementing mandatory access control',
 'Organizational processes for detection of unauthorized information system components',
 'Automated mechanisms supporting and/or implementing enforcement of adherence to protocol formats',
 'Automated mechanisms preventing automatic execution of unacceptable mobile code',
 'Automated mechanisms supporting and/or implementing safeguards for authenticator management',
 'Automated mechanisms implementing cross-organizational auditing (if applicable)',
 'Automated mechanisms implementing